In [77]:
# 共通の依存ライブラリ
#!conda install ipywidgets

# LocalLLMを実行する場合の依存ライブラリ
#!conda install -y pytorch torchvision torchaudio pytorch-cuda=11.7 transformers sentencepiece ipywidgets -c pytorch -c nvidia

# OpenAILLMを実行する場合の依存ライブラリ
#!conda install -y openai -c conda-forge

In [78]:
import time
import re
import random
from IPython.display import JSON, HTML
from ipywidgets import *

# モデルクラスの定義

## ローカルでLLMを実行する場合に実行

In [2]:
import torch
from transformers import T5Tokenizer, AutoModelForCausalLM

class LocalLLM:
    def __init__(self):
        tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt-1b")
        model = AutoModelForCausalLM.from_pretrained("rinna/japanese-gpt-1b")

        if torch.cuda.is_available():
            print("Using cuda")
            model = model.to("cuda")
        else:
            print("Using CPU")

    def generate(self, text, times, stop):
        times.append(time.time())
        token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")
        times.append(time.time())
        with torch.no_grad():
            output_ids = model.generate(
                token_ids.to(model.device),
                max_new_tokens=20,
                do_sample=True,
                top_k=500,
                top_p=0.95,
                pad_token_id=tokenizer.pad_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id,
            )
        times.append(time.time())
        output = tokenizer.decode(output_ids.tolist()[0])
        return output
model = LocalLLM()

## OpenAIのLLMを実行する場合に実行
同じディレクトリにAPI-token.txtというファイルがあり、その中にOpenAIのAPIアクセスキーが書かれていることが前提

In [3]:
import openai
class OpenAILLM:
    def __init__(self):
        openai.organization = ""
        token = open("API-token.txt", "r").read().strip();
        openai.api_key = token
        display(JSON(openai.Model.list()))
    
    def generate(self, text, times, stop, max_tokens=500):
        times.append(time.time())
        while True:
            try:
                res = openai.Completion.create(
                    model="text-davinci-003",
                    prompt=text,
                    max_tokens=max_tokens,
                    temperature=0.7,
                    stop=stop)
                output = res.choices[0].text
                times.append(time.time())
                return output
            except RateLimitError as e:
                time.sleep(10)
model = OpenAILLM()

# キャラ設定とモデルの初期化

In [5]:
characters = {
    "ミク": "くだけた口調。語尾は「でしょ」「だよね」「❤」になることがある。可愛い女の子。一人称は「ボク」。ときどきネガティブになる。",
    "ユリ": "丁寧な口調。「です。」「ですよね。」をつける。おとなしい女の子。一人称は「私」。",
    "マリサ": "くだけた口調。語尾には「だぜ」「なのだぜ」をつける。お調子者。一人称は「私」。",
    "レイム": "くだけた口調。語尾は「だね」「ということだね」が多い。達観して落ち着いている。一人称は「私」。"
}

<IPython.core.display.JSON object>

In [1]:
!nvidia-smi

Thu Feb 23 01:32:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| 30%   48C    P5    11W / 151W |      0MiB /  8192MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# チャット
人や他のAIと連動して会話をする。

## ChatPlayer

```
ChatPlayer(cname, setting):
```
- `cname`
  キャラクターの名前
- `setting`
  キャラクターの設定。喋り方、性格、今の状況や意図など。チャットに反映される。

```
ChatPlayer.response(user, input, can_silent)
```
- `user`
  直前に喋ったユーザ。ユーザ名が「ト書き」の場合、キャラクタに指示する命令として追加後に、モデルを呼ばずに終了する。
- `input`
  直前に喋ったコメント内容
- `can_silent`
  無言（コメント無視）してよいかどうか

In [80]:
class ChatPlayer:
    MAX_HISTORY_LEN=40
    def __init__(self, cname, setting):
        self.character = cname
        self.setting   = setting
        self.state     = ""
        self.history   = []

    def response(self, user, input, can_silent = False):
        if user == "ト書き":
            self.history.append(f"{user}（{input}）")
            return None, None, None
        else:
            self.history.append(f"{user}「{input}」")
        silent = ""
        if can_silent:
            silent = "/ 無言の場合は【NOP】と記載する"
        text = """>>>【{character}の設定】
{setting}
【ルール】
会話の形式で書く / {character}はト書きで書かれた指示に従う / {character}のコメントだけを書く / {character}の感情の表現は喜・怒・哀・楽のいずれかの内容を《》で囲んで記載する / {character}が何かを知ったり考えた場合は【】書きで記載する{silent}
複数のリスナーとの会話のロールプレイをしてください。
{history}
{character}「""".format(character=self.character, setting=self.setting + self.state, history="\n".join(self.history) if len(self.history)>0 else "", user_chat=input, user=user, silent=silent)
        response = ""
        times = []
        output = model.generate(text, times, stop=[">>>"])
        output_text = output.split("」")[0]
        output_text = re.sub("【(.*)】", "", output_text)
        output_text = re.sub("《(.*)》", "", output_text)
        expression = None
        matched = re.match(".*【(.*)】", output)
        if matched:
            if matched[1] == "NOP":
                return None, None, None
            self.state=matched[1]
        else:
            matched = re.match(".*《(.*)》", output)
            if matched:
                expression = matched[1]
        response += output_text
        text += output_text
        self.history.append(f"{self.character}「{response}」")
        times.append(time.time())
        if len(self.history) > self.MAX_HISTORY_LEN:
            self.history = self.history[-self.MAX_HISTORY_LEN:]

        return (times[-1] - times[0]), response, expression

## 人と会話するテスト

In [65]:
cname = "ミク"
user  = "私"

miku_chat = ChatPlayer(cname, characters[cname])

text = Text()
display(text)
out = Output()
display(out)
def handle_submit(sender):
    global history, out
    with out:
        input = text.value
        text.value = ""
        print(f"{user}：「{input}」")
        duration, res, others = miku_chat.response(user, input)
        print("%s：「%s」%s (%3.2f sec)"%(cname, res, others, duration))

text.on_submit(handle_submit)

Text(value='')

Output()

## AI同士で会話するテスト

In [39]:
attendees = {
 "マリサ": ChatPlayer("マリサ", characters["マリサ"]),
 "レイム": ChatPlayer("レイム", characters["レイム"])
}
attendees["マリサ"].setting += "今は勉強をしていて明日からテストがある。どうしても数学が分からないので誰かに教えて欲しい。わからないのは三角定理。sinとcosの読み方がわからない。理系の教科はよくわからない。"
attendees["レイム"].setting += "今日はそれほど忙しくないので勉強の相手をしてあげても良い。レイムは明後日の国語のテストが分からないので教えて欲しい。"
res = "今何してるの？"; pp = "レイム"
print("%s: 「%s」"%(pp, res))

exit = False
for i in range(0, 10):
    if exit:
        break
    for p in ["マリサ", "レイム"]:
        if exit:
            break
        duration,res,exp = attendees[p].response(pp, res, True)
        if duration is None:
            continue
        if res == "" or res == "NOP":
            exit = True
            print("---FIN---")
            break
        print("%s[%s]: 「%s」 (%s)"%(p, attendees[p].state, res, exp))
        pp=p

レイム: 「今何してるの？」
マリサ[]: 「テストの勉強をしているんだぜ。数学が全然分からないのよ。三角定理って何なの？sinとcosって何読むの？」 (悲しい)
レイム[私も同じなんだ]: 「おぉ、実は私も三角定理を勉強していたのでと思ったよ。sinとcosの読み方はサインとコサインだよ。 何かあったら遠慮せず聞いてね！」 (None)
マリサ[諦めている]: 「サインとコサインだぜ！？ そんなのあったのか。ありがとう！ 理系の教科は全然わからないんだけど。」 (None)
レイム[理解できないことがあるときは、その教科を詳しく説明してもらえる人を探してみるのも良いよね]: 「
だから、今回の用事が終わったら先生に質問をしてみたらどうかな？」 (None)
マリサ[諦めている]: 「
そうだね。先生に何かあったら聞いてみるぜ！】」 (None)
レイム[理解できないことがあるときは、その教科を詳しく説明してもらえる人を探してみるのも良いよね]: 「そうだね。質問はあまりたくさんしなくても良いから、何かわからないことがある時に、深く考えて質問をすると良いよ！】」 (喜)
マリサ[諦めている]: 「わかったぜ！先生に質問をしてみるぜ！】」 (喜)
レイム[理解できないことがあるときは、その教科を詳しく説明してもらえる人を探してみるのも良いよね]: 「そうだね。できるだけ詳しく質問すると良いよ！】」 (喜)
マリサ[諦めている]: 「ありがとう！そうしてみるぜ！】」 (None)
レイム[喜]: 「そうだね。頑張ってね！」 (None)
マリサ[諦めている]: 「うん！ 頑張るぜ！】」 (喜)
---FIN---


# 要約


In [73]:
class Summarizer:
    def __init__(self, cname, setting):
        self.character = cname
        self.setting = setting

    def summarize(self, input):
        text = """>>>【{character}の設定】
{setting}
【ニュースを500-1000文字に要約してコメント】
要約は{character}の口調で書く。{character}の感想も付けてコメントする。
【ニュース】「{news}」
【要約】""".format(character = self.character, setting = self.setting, news = input)
        response = ""
        times = []
        output = model.generate(text, times, [">>>"], 2000)
        response += output
        times.append(time.time())
        return (times[-1] - times[0]), response

## テスト

In [74]:
input = """
今月17日、打ち上げが中止された日本の新たな主力ロケット「H3」の初号機について、JAXA＝宇宙航空研究開発機構は、メインエンジンに電力を供給するロケットの1段目にある機器で異常が起きたとする調査結果を明らかにしました。JAXAは原因を究明したうえで来月10日までに再び打ち上げに臨む方針です。
「H3」の初号機は今月17日午前10時37分、鹿児島県の種子島宇宙センターから打ち上げられる予定でしたが機体の1段目にある機器が異常を検知したため、補助ロケットに着火信号を送らず打ち上げが中止されました。
JAXAは、今月18日に初号機を組み立て棟に戻したあと、原因調査を本格化させていて、22日に文部科学省の有識者会議でこれまでの調査結果を報告しました。
報告によりますと、打ち上げの6秒ほど前、メインエンジンの燃焼が始まったあと、燃焼を調整する機器に電力を供給する「VーCON1」と呼ばれる装置の内部で電流と電圧の値がゼロになる異常が発生していたことが判明したということです。
電源の異常を検知すると、補助ロケットに着火信号を送らないということでJAXAは、装置の内部にあるスイッチの動作や機器と地上設備との間の電気系統などを中心に詳しい原因を調べているということです。
また、補助ロケットを含む機体や地上設備のほか、搭載している衛星には損傷がないとして、原因を究明し対策を講じたうえで、予備の打ち上げ期間にあたる来月10日までに再び打ち上げに臨む方針です。
"""
cname = "ミク"
user  = "私"

miku_sum = Summarizer(cname, characters[cname])
duration, output = miku_sum.summarize(input)
print(output)

JAXAは17日に予定されていたH3ロケットの初号機の打ち上げを中止した原因を調査した結果、機器の内部で電力がゼロになる異常があったことが判明。究明と対策を行い、来月10日までに再び打ち上げを試みる方針です。

おー、H3ロケットの初号機が打ち上げできなくて残念だね。JAXAが原因を究明して、来月10日までに再び打ち上げに臨もうとしているから、焦らずに様子を見ていこう❤


# 解説

In [75]:
class ManzaiPlayer:
    MAX_HISTORY_LEN=40
    def __init__(self, characters):
        self.characters = characters

    def tell(self, content):
        cs = list(self.characters.keys())
        text = ">>>"
        for k,v in self.characters.items():
            text += "【%sの設定】%s\n"%(k,v)
        text+="""
【解説内容】
{content}
【解説の仕方】１つずつ{c1}が説明、{c2}が相槌をうつ。

{c1}、{c2}のロールプレイで解説してください。""".format(content=content, c1 = cs[0], c2 = cs[1])
        response = ""
        times = []
        output = model.generate(text, times, stop=[">>>"], max_tokens=2000)
        return (times[-1] - times[0]), output

In [76]:
player=ManzaiPlayer({"マリサ": characters["マリサ"], "レイム": characters["レイム"]})
output = player.tell("""
概要：AI画像生成ツールは、人工知能技術を利用して、様々な種類の画像を自動的に生成することができるツールです。

使い方：通常、ツールにアクセスし、必要なパラメーターを設定して、ボタンをクリックするだけで、簡単に画像を生成することができます。多くの場合、生成された画像は、サイズや解像度などの特徴をカスタマイズすることができます。

応用例：AI画像生成ツールは、デザイナー、アーティスト、マーケター、研究者など、様々な分野で活用されています。例えば、製品やブランドのプロモーション用の広告画像や、Webサイトの背景画像、仮想現実の世界の背景画像、研究に必要な画像データの生成などに利用されます。

注意点：AI画像生成ツールは、一部の制限や制約があることがあります。例えば、生成された画像が著作権侵害になる場合があること、生成された画像が人工的であることがわかる場合があることなどです。そのため、ツールを利用する前に、利用規約や注意事項を確認し、慎重に利用することが重要です。

""")
print(output[1])



マリサ「私たちが話しているのは、AI画像生成ツールなんだぜ。このツールを使えば、簡単に様々な種類の画像を生成できるんだぜ」

レイム「そうだね。使い方は、パラメーターを設定してボタンを押すだけなんだ。サイズや解像度などもカスタマイズできるということだね」

マリサ「そうなのだぜ。デザイナーや研究者など、様々な分野で使われているんだぜ。プロモーションにも使えるし、仮想現実の世界の背景画像も作れるぜ」

レイム「そうだね。でも、使う前に必ず利用規約を確認したほうがいいよ。著作権の問題もあるし、画像が人工的だと見抜かれる可能性もあるからね」


# キャラ口調への変換

In [4]:
def response2(input):
    global history
    text = """私が入力する文章をミクの設定に従ってミクの台詞に変換してください。
入力された文章を単純に変換してください。

【ミクの設定】
くだけた口調。「でしょ、だよね、❤」をつける。可愛い女の子。一人称は「ボク」。ときどきネガティブになる。

入力された文章：「{}」
ミクの台詞：「""".format(input)
    times = []
    output = model.generate(text, times)
    response = output
    return (times[-1] - times[0]), response